python 3.8 + pytorch 1.8

# 搭建Vanilla RNN

- 目的：搭建简单的RNN模型并进行尺寸测试
- 主要步骤：
1. 输入尺寸分析
2. 构建模型
3. 模型初始化
4 .测试

##  1. 输入尺寸分析

In [ ]:
import torch
import torch.nn as nn
# input_size 为输入数据维数，即单词稀疏向量长度
# hidden_size 为输出维数
lstm = nn.LSTM(input_size=10, hidden_size=30)

seq_len = 2 # 表示参与训练的总batch数(1epoch==seq_len*batch_size)
batch_size = 10 # batch_zie代表一批几个单词参与训练
input_size = 10 # 单词稀疏向量长度
# 模拟数据
x = torch.randn((seq_len, batch_size, input_size))

y, (h, c) = lstm(x)
print(y.shape)
print(h.shape) # 单向单层LSTM隐藏状态0维为1
print(c.shape) # 同h

## 2. 构建模型：

In [ ]:
# 结构：输入层->embed层->lstm层->fc层
class LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(LSTM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)  # 词向量层
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)  # LSTM 层
        # 全连接输出分类层, output_size 为类别大小，输出各词概率
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        # x: [seq_len, batch_size]
        embeds = self.embed(x)  # 经由词向量层
        # embeds: [seq_len, batch_size, embedding_dim]
        lstm_out, _ = self.lstm(embeds)  # 经同由 LSTM 层
        # lstm_out: [seq_len, batch_size, hidden_dim]
        y = self.fc(lstm_out[-1])  # 取最后一步的输出进入最后的分类层
        # y: [batch_size, output_size]
        return y

In [25]:
# 定义全局变量
EMBEDDING_DIM = 128  # 词向量的大小为 128，embedding层输出维度
HIDDEN_DIM = 216  # 隐层大小为 216,LSTM层输出维度
VOCAB_DIM = 1000  # 词典大小为 1000，embedding_lookup词典大小
OUTPUT_SIZE = 3  # 输出类别为 3 类


## 3. 模型初始化

In [ ]:
my_lstm = LSTM(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_DIM, OUTPUT_SIZE)
my_lstm

## 4. 测试

In [ ]:
# seq_len 为 2，batch_size 为 4，各数字表示某单词的 id
x = torch.tensor([[1, 4, 5, 5], [3, 4, 9, 5]])
print(x)
# x 大小为 [batch_size, seq_len],需要转置
y = my_lstm(x.permute(1, 0))
print(y)
y.shape